In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

from importlib import reload
#syntax: fugi = reload(fugi)

import functions_giammi as fugi
# fugi = reload(fugi)

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline
%config InlineBackend.figure_formats=["png"]

import re

#for the cartesian product
import itertools

#to smooth the MFPADs
import scipy as sp
import scipy.ndimage

#to create the mesh
import triangulation as tr
from scipy.spatial import Delaunay
from itertools import product
from pyntcloud import PyntCloud, structures

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

#loads the temperaure cmap like Philipp for both matpplotlib and plotly
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()

#to see which classes have been defined so far
[modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

###550eV - 11.5eV CH11
#update nov Kilian binning (36,18)
fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH11_550eV_CR_Knov.root");en="550eV";CH="CH11";
fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH11_550eV_CL_Knov.root")
fileSCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH11_550eV_CR_Knov.root")
fileSCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH11_550eV_CL_Knov.root")

#Select between CH9 and CH11
path = "angular_distr_el/"+CH+"/"

#move first the ID you need
IDs = ["ID_ALL_mol_e0_valid/EN_gate/","ID1_mol_e0_valid/EN_gate/","ID2_mol_e0_valid/EN_gate/","ID3_mol_e0_valid/EN_gate/"]
dirs = ["MFPADs_multinew/","MFPADs_multinew_phi/"]

loc=path+IDs[0]+dirs[0]

#load theoretical data
pathth = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=11.5eV';en="KE=11.5eV";

# glob has to be sorted (both in unix and windows)
all_files = sorted(glob.glob(pathth + "/**.dat"))

li = []
cosphi_th = []
test_opening = []
allnames=[]
colnames = ["phi", "cos(theta)", "value"]

# how to load multiple files http://jonathansoma.com/lede/foundations-2017/classes/working-with-many-files/class/,
for filename in all_files:
    df = pd.read_csv(filename, delimiter=r"\s+", names=colnames, header=None)  # r"\s+" is a regex (regular expression)
    # or delim_whitespace=True, it is faste
    temp_name=(filename.replace(" ","")).split("/")[-1].split(".")[0] # adding a column to df with the file name with no spaces and extension
    cosphi_th.append((int(temp_name.split("_")[0]), int(temp_name.split("_")[1])))
    allnames.append(temp_name)
    df["filename"]=temp_name
    df["filename"]=df["filename"].astype("category")
    li.append(df) # a unique DataFrame

cosphi_th = np.array(cosphi_th)
cosphi_th_adj = np.around(np.array(list(itertools.product(np.linspace(1.,-1.,6).tolist(),np.linspace(-180,180,12).tolist()))),3)
cosphi_th_adj_exp = np.around(np.array(list(itertools.product(np.linspace(0.835,-0.835,6).tolist(),np.linspace(-165,165,12).tolist()))),3)

frame = pd.concat(li, axis=0)
#this commmand avoids 1_1 followed by 1_10: semantic order
frame_srt = frame.groupby("filename",sort=False)
frame_set = frame.set_index("filename") # build a multiindex using the categories of filename

In [ ]:
#pick the right theoretical MFPAD and inver the cos(theta)_el axis
counts_temp=[]
for group in frame_srt:
    # elsorted=group[1].sort_values(by=["cos(theta)","phi"]) #just a try to remap cos(theta)
    # counts_temp.append(elsorted["value"].values.reshape(200,100))
    counts_temp.append(group[1]["value"].values)
counts_temp=np.array(counts_temp).reshape(72,200,100)

#NOTE here the cos(theta)_el is already sorted as in MFPAD_Sth_phi [-1:-1]
phi_full_sorted=np.array(frame_srt.get_group("1_1").sort_values(by=["cos(theta)","phi"])["phi"])
ctheta_full_sorted=np.array(frame_srt.get_group("1_1").sort_values(by=["cos(theta)","phi"])["cos(theta)"])

MFPAD_Sth_phi=fugi.sorting_array(counts_temp, theory=True, items=cosphi_th, a=2)

print("MFPAD th shape ", MFPAD_Sth_phi.shape)


In [ ]:
#Loading of the experimental MFPADS
MFPAD_RCR,ctheta_RCR,ctheta_RCR_err,cosphi_photon,MFPAD_axis,ctheta_axis,ctheta_axis_red=fugi.import_MFPAD(fileRCR, loc, full=True)
MFPAD_RCL,ctheta_RCL,ctheta_RCL_err=fugi.import_MFPAD(fileRCL, loc)

MFPAD_SCR,ctheta_SCR,ctheta_SCR_err=fugi.import_MFPAD(fileSCR, loc)
MFPAD_SCL,ctheta_SCL,ctheta_SCL_err=fugi.import_MFPAD(fileSCL, loc)

MFPAD_RCR_norm=fugi.normalise_with_err(MFPAD_RCR)
MFPAD_RCL_norm=fugi.normalise_with_err(MFPAD_RCL)
MFPAD_SCR_norm=fugi.normalise_with_err(MFPAD_SCR)
MFPAD_SCL_norm=fugi.normalise_with_err(MFPAD_SCL)

#COORDIANTES PHOTON ROTATION
#adjusting cosphi with a shifting to have it equal to theory
ctheta_axis_red=ctheta_axis_red[0]; #Ffrom ccos(theta)

phiM=(MFPAD_axis[0][0][1:] + MFPAD_axis[0][0][:-1])/2 #phi = (36,), MFPAD_axis = (37,)
cosM=(MFPAD_axis[0][1][1:] + MFPAD_axis[0][1][:-1])/2 #COS = (18,), MFPAD_axis = (19,)
phiMM, cosMM = np.meshgrid(phiM, cosM)

phicos_phi=list(product(phiM, cosM))
phi_full = np.array([el[0] for el in phicos_phi])
cos_full = np.array([el[1] for el in phicos_phi])

phi_rad= phi_full*np.pi/180.
theta_rad= np.arccos(cos_full)

print("MFPAD_xy shape = ", MFPAD_axis[0][0].shape)
print("len phi_rad = ", phi_rad.shape)
print("len theta_rad = ", theta_rad.shape)
print("phiM shape = ", phiM.shape)
print("cosM shape = ", cosM.shape)

In [ ]:
plt.imshow(MFPAD_SCL_norm[0].T, cmap="viridis")

In [ ]:
#down sample theoretical MFPAD to 18 36
from scipy.interpolate import griddata

temp=[]
for el in MFPAD_Sth_phi:
    temp.append(griddata(list(zip(phi_full_sorted.reshape(-1),ctheta_full_sorted.reshape(-1))), el.reshape(-1), (phiMM, cosMM), method="linear"))

MFPAD_Sth_phi_red=np.array(temp).reshape(72,36,18)

In [ ]:
plt.imshow(MFPAD_Sth_phi_red[0].T, cmap="viridis")

In [ ]:
#describe a set of alpha, beta, gamma angles to rotate the theoretical distribution (would be better to rotate before downsampling)
arr=np.arange(-30,30,5)
#alpha, beta, gamma for 3D rotation
abg=list(product(arr, arr, arr))

a = np.array([el[0] for el in abg])
b = np.array([el[1] for el in abg])
g = np.array([el[2] for el in abg])

In [ ]:
# rotate the theoretical distribution

# these ae now 1728 MFPAD (18,36) of the first MFPAD
MFPAD_rotated=fugi.rot3d_MFPAD_dist(MFPAD_Sth_phi_red[0],theta_rad,phi_rad,abg,phiMM,cosMM)

MFPAD_rotated_norm=fugi.normalise_with_err(MFPAD_rotated)

In [ ]:
#difference square looping along all coordiantes: NOTE they should have filled int he same way
#normalize before
d_temp=[]
for el in MFPAD_rotated_norm:
    d_temp.append(np.sum((MFPAD_SCL_norm[0].reshape(-1) - el.reshape(-1)))**2)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

img = ax.scatter(a, b, g, c=d_temp, alpha=0.5, cmap=plt.hot())
fig.colorbar(img)
plt.show()